In [8]:
from bs4 import BeautifulSoup
import requests
import pandas as pd
import numpy as np
import re
import time, os
from selenium import webdriver
from selenium.webdriver.common.keys import Keys

from webdriver_manager.chrome import ChromeDriverManager

import requests

import math

import json


In [9]:
url = "http://www.gsmarena.com/makers.php3"
root_url= "http://www.gsmarena.com/"
information = []

In [10]:
mainpage = requests.get(url)
soup = BeautifulSoup(mainpage.text,'html.parser')


In [11]:
table = soup.find('div', class_="st-text")
links = table.find_all('a')

In [12]:
pagelink = [a.attrs.get('href') for a in links]

In [13]:
def get_links(href):
     page = requests.get(root_url + str(href))
     soup = BeautifulSoup(page.text,'html.parser')
     table = soup.find('div',class_="section-body")
     links = table.find_all('a')
     return [a.attrs.get('href') for a in links]
    
#get phone info from 1 page
def phoneinfo(directlink):
     mainlink = root_url + str(directlink)
     response = requests.get(mainlink)
     soup = BeautifulSoup(response.content,'html.parser')
     currentdate = time.strftime("%c")
     name = soup.find("h1", class_="specs-phone-name-title").get_text()
     specs = soup.find("div", id="specs-list")
     table = specs.find_all("table")
     dimension = table[2].get_text()
     print("Phone Name:" + " " + name)
     d=[]
     for t in dimension.split():
         try: 
            d.append(float(t))
         except ValueError:
            pass
     screen = table[3].get_text()
     S=[]
        # to collect only numbers from the screen text
     for b in screen.split():
        try:
             S.append(float(b))
        except ValueError:
             pass
        if len(d) < 4 and len(S) > 2 :
                height = "-"
                width = "-"
                inch = S[0]
                mm = round (float(S[0])*25.4,2)
                pixWidth = S[1]
                pixHeight = S[2]
                ratio = round(int(S[1])/int(S[2]),3)
                if 0.6 > ratio >=0.562:
                         ratioW = 9
                         ratioH = 16
                elif 0.625> ratio >= 0.6:
                         ratioW = 3
                         ratioH = 5
                elif 0.667> ratio >= 0.625:
                         ratioW = 10
                         ratioH = 16
                elif 0.75> ratio >= 0.667:
                         ratioW = 2
                         ratioH = 3
                elif 0.8 > ratio >= 0.75:
                         ratioW = 3
                         ratioH = 4
                else:
                         ratioW = 4
                         ratioH = 5
                         screenW = round(math.sqrt((math.pow(mm,2) *
                         math.pow(ratioW,2))/(math.pow(ratioW,2)+math.pow(ratioH,2))),2)
                         screenH = round(math.sqrt((math.pow(mm,2) - math.pow(screenW,2))),2)
                         sidebezel = "-"
                         topbotbezel = "-"
                         status = "Data Incomplete"
        elif len(d) >= 4 and len(S)<=2:
                 height = d[0]
                 width = d[1]
                 inch = "-"
                 mm = "-"
                 pixWidth = "-"
                 pixHeight = "-"
                 ratioW = "-"
                 ratioH = "-"
                 screenH = "-"
                 screenW = "-"
                 sidebezel ="-"
                 topbotbezel = "-"
                 status = "Data Incomplete"
        elif len(d) < 4 and len(S)<=2:
                 height = "-"
                 width = "-"
                 inch = "-"
                 mm = "-"
                 pixWidth = "-"
                 pixHeight = "-"
                 ratioW = "-"
                 ratioH = "-"
                 screenW ="-"
                 screenH = "-"
                 sidebezel = "-"
                 topbotbezel = "-"
                 status = "Data Incomplete"
        else:
                height = d[0]
                width = d[1]
                inch = S[0]
                mm = round (float(S[0])*25.4,2)
                pixWidth = S[1]
                pixHeight = S[2]
                ratio = round(int(S[1])/int(S[2]),3)
                if 0.6 > ratio >=0.562:
                         ratioW = 9
                         ratioH = 16
                elif 0.625> ratio >= 0.6:
                         ratioW = 3
                         ratioH = 5
                elif 0.667> ratio >= 0.625:
                         ratioW = 10
                         ratioH = 16 
                elif 0.75> ratio >= 0.667:
                         ratioW = 2
                         ratioH = 3
                elif 0.8 > ratio >= 0.75:
                         ratioW = 3
                         ratioH = 4
                else:
                         ratioW = 4
                         ratioH = 5
                screenW = round(math.sqrt((math.pow(mm,2) * math.pow(ratioW,2))/(math.pow(ratioW,2)+math.pow(ratioH,2))),2)
                screenH = round(math.sqrt((math.pow(mm,2) - math.pow(screenW,2))),2)
                sidebezel = round((d[1]- screenW) / 2, 2)
                topbotbezel = round((d[0] - screenH) / 2, 2)
                status = "Data Complete"
    # print all information to double check
        print("Phone Height:"+ " " + str(height))
        print("Phone Width:" + " " + str(width))
        print("Phone Diagonal in inch:" + " " + str(inch))
        print("Phone Diagonal in mm:" + " " + str(mm))
        print("Aspect Ratio=" +str(ratioW)+":"+str(ratioH))
        print("Pixel Width:" +" " + str(pixWidth))
        print("Pixel Height:" + " " + str(pixHeight))
        print("Screen Width:" + " " + str(screenW))
        print("Screen Height:" + " " + str(screenH))
        print("Side Bezels:" + " "+ str(sidebezel))
        print("Top and Bottom Bezel:" + " "+ str(topbotbezel))
        print("")
        entry ={"name": name, "phoneH": height,"phoneW": width,"sizeinch": inch,"sizemm": mm,
                 "pixelW": pixWidth,"pixelH": pixHeight,"ratioW": ratioW,"ratioH": ratioH,
                 "screenW":screenW,"screenH": screenH,"sideB": sidebezel,"tbbezel": topbotbezel,
                 "status":status,"Date Taken":currentdate, "Taken From":mainlink}
        information.append(entry)
        time.sleep(2)             

In [14]:
# collect nav page info
def navpage(href):
     response = requests.get(root_url+str(href))
     soup = BeautifulSoup(response.text,'html.parser')
     table = soup.find('div', class_="nav-pages")
     links = table.find_all('a')
     return [a.attrs.get('href') for a in links]

In [15]:
# program to start getting data from the 1st phone brand
def start(href): 
     links = get_links(href)
     for directlink in links:
             phoneinfo(directlink)
             try:
                with open('Phone Database.txt','w') as p:
                      json.dump(information, p, indent=1)
            
             except:
                continue
                time.sleep(2)
     time.sleep(2)

In [16]:
# main program
def main():
    for href in pagelink:
         start(href)
         try:
            extrapages = navpage(href)
            for directlink in extrapages:
                   start(directlink)
         except:
             Continue
             time.sleep(2)
print(main()) 

Phone Name: Acer Chromebook Tab 10
Phone Height: 238.3
Phone Width: 172.2
Phone Diagonal in inch: -
Phone Diagonal in mm: -
Aspect Ratio=-:-
Pixel Width: -
Pixel Height: -
Screen Width: -
Screen Height: -
Side Bezels: -
Top and Bottom Bezel: -

Phone Height: 238.3
Phone Width: 172.2
Phone Diagonal in inch: -
Phone Diagonal in mm: -
Aspect Ratio=-:-
Pixel Width: -
Pixel Height: -
Screen Width: -
Screen Height: -
Side Bezels: -
Top and Bottom Bezel: -

Phone Height: 238.3
Phone Width: 172.2
Phone Diagonal in inch: -
Phone Diagonal in mm: -
Aspect Ratio=-:-
Pixel Width: -
Pixel Height: -
Screen Width: -
Screen Height: -
Side Bezels: -
Top and Bottom Bezel: -

Phone Height: 238.3
Phone Width: 172.2
Phone Diagonal in inch: -
Phone Diagonal in mm: -
Aspect Ratio=-:-
Pixel Width: -
Pixel Height: -
Screen Width: -
Screen Height: -
Side Bezels: -
Top and Bottom Bezel: -

Phone Height: 238.3
Phone Width: 172.2
Phone Diagonal in inch: -
Phone Diagonal in mm: -
Aspect Ratio=-:-
Pixel Width: -
Pixe

AttributeError: 'NoneType' object has no attribute 'get_text'